In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

def get_web(link):
    resp = requests.get(link) 
    html_code = resp.text 
    soup = BeautifulSoup(html_code, 'html.parser') 
    return soup

In [2]:
resp = requests.get('https://ca.trustpilot.com/review/tdbank.ca')
print(type(resp))
print(resp.status_code)

<class 'requests.models.Response'>
200


In [3]:
with open('page.html', 'w') as f:
    f.write(resp.text)
#question 1: saving as html

In [4]:
link = 'https://ca.trustpilot.com/review/tdbank.ca' 
soup = get_web(link)
print(soup.title)

div_complete_contents = soup.find_all("div", {"class": "review-content"}) 
print(div_complete_contents)

<title>TD Bank Reviews | Read Customer Service Reviews of tdbank.ca</title>
[<div class="review-content">
<div class="review-content__header" v-pre="">
<div class="review-content-header">
<div class="star-rating star-rating--medium">
<img alt="1 star: Bad" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-1.svg"/>
</div>
<div class="review-content-header__dates">
<script data-initial-state="review-dates" type="application/json">
{"publishedDate":"2021-05-26T05:02:20+00:00","updatedDate":null,"reportedDate":null}
</script>
<review-dates :published-date="publishedDate" :reported-date="reportedDate" :updated-date="updatedDate"></review-dates>
</div>
<div class="review-content-header__review-labels">
</div>
</div>
</div>
<div class="review-content__body" v-pre="">
<h2 class="review-content__title">
<a class="link link--large link--dark" data-track-link="{'target': 'Single review', 'name': 'review-title'}" href="/reviews/60add65cf9f4870a94ad2e5f">
                        Td ban

In [5]:
company_name = soup.find("span",{"class":"multi-size-header__big"}).text 
print(company_name)
count = int(soup.find("span",{"class":"headline__review-count"}).text) 
print(count)
count_per_page = len(div_complete_contents) 
pages = (count//count_per_page)+1
print(pages)
#question 2: total number of reviews: there are 859 reviews and 43 pages of reviews


                        TD Bank
                    
861
44


In [6]:
#question 3: iterate
total_data = []

pages = np.arange(1, 44, 1)

for page in pages:
    page = requests.get('https://ca.trustpilot.com/review/tdbank.ca' + "?page=" + str(page))

    soup = BeautifulSoup(page.text, "html.parser")

#question 4: extract reviews (also above after question 1)

    
    div_complete_contents = soup.find_all("div", {"class": "review-content"})
    print(div_complete_contents)

    for num, div_content in enumerate(div_complete_contents):
   
#question 5: storing date, rating, and review body to review_info and then adding it to total_data
        date_data = div_content.find_all("script") 
        date_cts = date_data[0].__dict__["contents"]
        date = date_cts[0].split('"publishedDate":"')[1][:10]

       
        review_data = div_content.find_all("img")
        atr = review_data[0].__dict__["attrs"]
        rating = atr["alt"][0]
        review_body = div_content.find("p",{"class":"review-content__text"}).text 
        review_info = [company_name,date,rating,review_body]
        total_data.append(review_info)


Streaming output truncated to the last 5000 lines.
<div class="review-content-header">
<div class="star-rating star-rating--medium">
<img alt="1 star: Bad" src="https://cdn.trustpilot.net/brand-assets/4.1.0/stars/stars-1.svg"/>
</div>
<div class="review-content-header__dates">
<script data-initial-state="review-dates" type="application/json">
{"publishedDate":"2021-05-20T00:26:35+00:00","updatedDate":null,"reportedDate":null}
</script>
<review-dates :published-date="publishedDate" :reported-date="reportedDate" :updated-date="updatedDate"></review-dates>
</div>
<div class="review-content-header__review-labels">
</div>
</div>
</div>
<div class="review-content__body" v-pre="">
<h2 class="review-content__title">
<a class="link link--large link--dark" data-track-link="{'target': 'Single review', 'name': 'review-title'}" href="/reviews/60a5acbbf9f4870a94a78d84">
                        Non professional people TD…
                </a>
</h2>
<p class="review-content__text">
                Wha

In [7]:
df = pd.DataFrame(total_data, columns=["companyName","datePublished","ratingValue","reviewBody"])
df.to_csv("TD reviews final.csv", index=False)
#question 6: final csv 